## Load Modules

In [1]:
import torch
import pandas as pd
from efficientnet_pytorch import EfficientNet
from datetime import date
import glob
from PIL import Image
from torchvision import  transforms,datasets, models
import torch.nn as nn
import sklearn

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Private variables

In [2]:
PATH_WEIGHTS = "D:/ML_exercise/Weights"
#PUBLIC_TESTING_PATH = "/work/u2785625/AI_Cup_2022/ProcessedPublicTesting/"
folderNames = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
OUTPUT_CSV_PATH = "D:/ML_exercise/PublicTesting_OutputCsv"

#local_dir = "/work/u2785625/AI_Cup_2022/Datasets"
local_dir = "D:/ML_exercise"
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
species = ['asparagus', 'bambooshoots', 'betel', 'broccoli', 'cauliflower', 'chinesecabbage', 'chinesechives', 'custardapple', 'grape', 'greenhouse', 'greenonion', 'kale', 'lemon', 'lettuce', 'litchi', 'longan', 'loofah', 'mango', 'onion', 'others', 'papaya', 'passionfruit', 'pear', 'pennisetum', 'redbeans', 'roseapple', 'sesbania', 'soybeans', 'sunhemp', 'sweetpotato', 'taro', 'tea', 'waterbamboo']

## Testing Transformation

In [4]:
test_tfm = transforms.Compose([ #valid and grading
    transforms.ToTensor(),
    transforms.Resize((456,456)),
    transforms.CenterCrop(456),
    transforms.Normalize([.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
])

#data = datasets.ImageFolder(root = local_dir, transform = test_tfm)

## Load Model

In [5]:
model = models.efficientnet_b5(weights='DEFAULT').to(device)
num_ftrs = model.classifier[1].in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.load_state_dict(torch.load(PATH_WEIGHTS))


<All keys matched successfully>

In [6]:
print(num_ftrs)
print(n_classes)

2048
33


## Prediction

In [7]:
model.eval()
model.to(device)
IMG_PATH = "D:/ML_exercise/sunhemp/ffd41947-cac2-4e0b-8e58-e6bfa49019c1.jpg"
Final_label_index = [1, 2, 10, 11, 16, 17, 19, 20, 29, 30]
PAR = 0.1
img = Image.open(IMG_PATH)
x = test_tfm(img).to(device)
x = x.unsqueeze(0)
output = model(x)
#print(output)
_, pred = torch.max(output, dim = 1)
percentage = nn.functional.softmax(output, dim=1)[0]
perc = percentage[int(pred)].item()
print(perc)
result = species[pred]
print(result)

# 得到预测结果，并且从大到小排序
_, indices = torch.sort(output, descending=True)
# 返回每个预测值的百分数
print([(species[idx], percentage[idx].item()) for idx in indices[0][:9]])


0.8622316122055054
sunhemp
[('sunhemp', 0.8622316122055054), ('sesbania', 0.11614012718200684), ('others', 0.016279706731438637), ('bambooshoots', 0.003470506053417921), ('waterbamboo', 0.0014112989883869886), ('asparagus', 0.0004125163541175425), ('sweetpotato', 1.946224074345082e-05), ('pennisetum', 1.370804511680035e-05), ('taro', 6.687555924145272e-06)]


In [13]:
for i in range(len(Final_label_index)):
    output[0][Final_label_index[i]] += PAR
#print(output)
_, pred = torch.max(output, dim = 1)
percentage = nn.functional.softmax(output, dim=1)[0]
perc = percentage[int(pred)].item()
print(perc)
result = species[pred]
print(result)

# 得到预测结果，并且从大到小排序
_, indices = torch.sort(output, descending=True)
# 返回每个预测值的百分数
print([(species[idx], percentage[idx].item()) for idx in indices[0][:9]])

tensor([28], device='cuda:0')
0.8563043475151062
sunhemp
[('sunhemp', 0.8563043475151062), ('sesbania', 0.11534174531698227), ('others', 0.021824266761541367), ('bambooshoots', 0.004652494098991156), ('waterbamboo', 0.0014015971682965755), ('asparagus', 0.0004096805932931602), ('sweetpotato', 2.6090710889548063e-05), ('pennisetum', 1.3613811461254954e-05), ('taro', 8.965202141553164e-06)]


In [10]:
print(pred)

tensor([28], device='cuda:0')


In [14]:

model.eval()
model.to(device)
List_FileName = list()
List_Output = list()
for folder in folderNames:
    folderPath =  str(local_dir) + '/'  + folder + '/'
    for fileName in glob.glob(folderPath + '*'):
        print(fileName)
        img = Image.open(fileName)
        x = test_tfm(img)
        x = x.unsqueeze(0)
        output = model(x)    
        for i in range(len(Final_label_index)):
            output[0][Final_label_index[i]] += PAR   
        _, pred = torch.max(output, dim = 1)
        print(pred)
        print(fileName[len(folderPath)::])
        print(species[pred])        
        List_FileName.append(fileName[len(folderPath)::])
        List_Output.append(species[pred])

In [6]:
output_dict = {"filename": List_FileName,
                "label": List_Output}
Out_dataframe = pd.DataFrame(output_dict)
print(Out_dataframe)

                                       filename           label
0      09462dbc-ecc4-45d7-8653-9e3a22bbacc2.jpg          longan
1      059b89e1-80c1-43a4-be36-cc911dc440f9.jpg      greenhouse
2      0ea762fd-124b-4db5-827a-60799a869870.jpg  chinesecabbage
3      0e03f42d-70b7-4671-aefe-3e7402ac90a9.jpg     cauliflower
4      0f8b5ffa-8609-4e05-afc0-56717bc0c113.jpg    bambooshoots
...                                         ...             ...
11143  f06bbcf8-f784-4810-92b9-c249c93f3f81.jpg            kale
11144  f2efc7a7-48a6-429a-be7f-86884826f223.jpg    bambooshoots
11145  fab7c542-6cc0-410c-8b33-8a9e0675029a.jpg      greenhouse
11146  fe0231e7-fdf1-4415-b586-e0e815b39ba4.jpg            kale
11147  fdc8142c-74ea-4cfe-baf8-bd9c70ed0da7.jpg     sweetpotato

[11148 rows x 2 columns]


## Save Csv

In [7]:
FileName =  date.today().strftime("%m/%d").replace('/','') + '.csv'

Out_dataframe.to_csv(OUTPUT_CSV_PATH + FileName , index = False)